In [1]:
# !pip install splinter
# !pip install selenium

In [2]:
# Import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser

import pandas as pd
import requests
import time

In [3]:
# For MAC users
!which chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


# NASA Mars News

In [4]:
# URL of NASA Mars News Site to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(5)

In [5]:
# HTML object
html = browser.html
    
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
    
# Retrieve the latest news title and paragraph text and assign correponding variables
news = soup.find('div', class_='list_text')
title = news.find('div', class_='content_title').text
text = news.find('div', class_='article_teaser_body').text

print (f"NEWS TITLE:\n{title}\n")
print (f"PARAGRAPH TEXT:\n{text}")

NEWS TITLE:
Digital Creators: Apply for NASA Mars Landing Event

PARAGRAPH TEXT:
Up to 30 multimedia participants will be selected for a special two-day event at JPL culminating in the InSight spacecraft's landing on Mars. Apply by Sept. 3.


# JPL Mars Space Images - Featured Image

In [20]:
# URL of JPL Featured Space Image to be scraped using Splinter
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Click on "Full Image" button
browser.find_by_id('full_image').click()

# HTML object
html = browser.html
    
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Get relative path of full image
image_url = soup.find("a", class_="button fancybox")['data-fancybox-href']

# Create variable for base URL 
base_url = 'https://www.jpl.nasa.gov'

# Create full URL for featured image
featured_image_url = base_url + image_url

print (f"FEATURED IMAGE URL:\n{featured_image_url}\n")


FEATURED IMAGE URL:
https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA09178_ip.jpg



# Mars Facts

In [7]:
# Visit the Mars Facts Webpage
url = 'http://space-facts.com/mars/'
browser.visit(url)

# Use Pandas to scrape the table containing Mars facts
table = pd.read_html(url)

# Select table
mars_facts_DF = table[0]

# Format table by adding column names and resetting index
mars_facts_DF.columns = ['Description', 'Values']
mars_facts_DF = mars_facts_DF.set_index('Description')

# Print Mars Facts Table
mars_facts_DF

,Values
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [8]:
# Use Pandas to convert the data to a HTML table string
mars_table = mars_facts_DF.to_html().replace('\n', ' ')
mars_table


'<table border="1" class="dataframe">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Values</th>     </tr>     <tr>       <th>Description</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

# Mars Hemispheres

In [21]:
# Visit the USGS Astrogeology site 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

# HTML object
html = browser.html
    
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Create list for storing image URLs and hemisphere titles
hemisphere_image_urls = []

# Loop through all products and get relevant information
for x in range(4):
    
    # Identify item link and click on it
    item = browser.find_by_tag('h3')
    item[x].click()
    
    # Create HTML browser object and parse with Beautiful Soup
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Get hemisphere title and full image URL
    img_title = soup.find('h2', class_='title').text
    img_url = soup.find('a', target='_blank')['href']
    
    # Create dictionary and append to list storing all hemisphere titles and image URLs.
    dictionary = {"title":img_title,"img_url":img_url}
    hemisphere_image_urls.append(dictionary)
    
    # Click on Back button to return to previous site
    browser.back()

print('Hemisphere & Image URLs:')
print('------------------------')
print(hemisphere_image_urls)


Hemisphere & Image URLs:
------------------------
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
